In [36]:
# 基础库
import os
import numpy as np
import pandas as pd

# 地理与空间数据处理
import geopandas as gpd

# 可视化
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
import seaborn as sns
from matplotlib.colors import BoundaryNorm, ListedColormap, LogNorm
from matplotlib.patches import Wedge
from matplotlib.collections import PatchCollection
import matplotlib.font_manager as fm
import statsmodels.api as sm

# 调色板（可选）
import palettable

# Excel 支持
import openpyxl

#检查当前工作目录
print(os.getcwd())

#修改当前工作目录
os.chdir("/public/home/jiaqizhang/JiaqiZhang/project-CCFR/02.Analysis and figures/01.notebook/Analyse2.Excel_subcountry_ranking")

#检查当前工作目录
print(os.getcwd())

/public/home/jiaqizhang/JiaqiZhang/project-CCFR/02.Analysis and figures/01.notebook/Analyse2.Excel_subcountry_ranking
/public/home/jiaqizhang/JiaqiZhang/project-CCFR/02.Analysis and figures/01.notebook/Analyse2.Excel_subcountry_ranking


In [37]:
#------------------------------------------打开次国家边界的shp文件----------------------------------------

# 读取边界文件
subcountry = gpd.read_file("/public/home/jiaqizhang/JiaqiZhang/data-doctor3/country/ne_10m_admin_1_states_provinces.shp", engine="pyogrio")

# 过滤掉 code_hasc 以 ~ 结尾的行
subcountry = subcountry[~subcountry['code_hasc'].str.endswith('~', na=False)]

# 筛选台湾地区（以 "TW.TW." 开头的 code_hasc）
taiwan = subcountry[subcountry['code_hasc'].str.startswith('TW.')]
# 合并台湾的各行政区为一个整体边界
taiwan_merged = taiwan.dissolve(by=lambda x: 'TW')
# 添加一个字段表示国家代码
taiwan_merged['country'] = 'CN'
taiwan_merged['name'] = 'Taiwan'
taiwan_merged['code_hasc'] = 'CN.TW'
print(taiwan_merged['code_hasc'])  # 或 'hasc_code'，看原始字段名
# 如果你想将这个合并结果加入原始数据（并删除原本的台湾省份）
boundary_no_tw = subcountry[~subcountry['code_hasc'].str.startswith('TW.')]
subcountry = pd.concat([boundary_no_tw, taiwan_merged], ignore_index=True)


# 筛选香港地区（以 "HK." 开头的 code_hasc）
hongkong = subcountry[subcountry['code_hasc'].str.startswith('HK.')]
# 合并香港的各行政区为一个整体边界
hongkong_merged = hongkong.dissolve(by=lambda x: 'HK')
# 添加一个字段表示国家代码
hongkong_merged['country'] = 'CN'
hongkong_merged['name'] = 'Hongkong'
hongkong_merged['code_hasc'] = 'CN.HK'
print(hongkong_merged['code_hasc'])  # 或 'hasc_code'，看原始字段名
# 如果你想将这个合并结果加入原始数据（并删除原本的台湾省份）
boundary_no_hk = subcountry[~subcountry['code_hasc'].str.startswith('HK.')]
subcountry = pd.concat([boundary_no_hk, hongkong_merged], ignore_index=True)

print(len(subcountry))
print(subcountry.columns.tolist())
print(subcountry.head)

# 步骤 6️⃣：将结果保存为 Excel 文件，便于分析或绘图
output_path = 'name.xlsx'
subcountry.to_excel(output_path, index=False)

#------------------------------------------打开国家边界的shp文件----------------------------------------
continent = gpd.read_file("/public/home/jiaqizhang/JiaqiZhang/data-doctor3/country/ne_10m_admin_0_countries_chn.shp", engine="pyogrio")

# # 打印前5行数据查看
# print(continent_gdf.head())

# # Print all column names
# print(continent_gdf.columns.tolist())

# print(len(continent_gdf))

TW    CN.TW
Name: code_hasc, dtype: object
HK    CN.HK
Name: code_hasc, dtype: object
4457
['featurecla', 'scalerank', 'adm1_code', 'diss_me', 'iso_3166_2', 'wikipedia', 'iso_a2', 'adm0_sr', 'name', 'name_alt', 'name_local', 'type', 'type_en', 'code_local', 'code_hasc', 'note', 'hasc_maybe', 'region', 'region_cod', 'provnum_ne', 'gadm_level', 'check_me', 'datarank', 'abbrev', 'postal', 'area_sqkm', 'sameascity', 'labelrank', 'name_len', 'mapcolor9', 'mapcolor13', 'fips', 'fips_alt', 'woe_id', 'woe_label', 'woe_name', 'latitude', 'longitude', 'sov_a3', 'adm0_a3', 'adm0_label', 'admin', 'geonunit', 'gu_a3', 'gn_id', 'gn_name', 'gns_id', 'gns_name', 'gn_level', 'gn_region', 'gn_a1_code', 'region_sub', 'sub_code', 'gns_level', 'gns_lang', 'gns_adm1', 'gns_region', 'min_label', 'max_label', 'min_zoom', 'wikidataid', 'name_ar', 'name_bn', 'name_de', 'name_en', 'name_es', 'name_fr', 'name_el', 'name_hi', 'name_hu', 'name_id', 'name_it', 'name_ja', 'name_ko', 'name_nl', 'name_pl', 'name_pt', '

In [38]:
##------------------------------------------打开gridnet洪水数据的shp文件----------------------------------------

shp_file_path = "/public/home/jiaqizhang/JiaqiZhang/project-CCFR/01.Processed data/11.GDP/gridnet_GDP_CCFR.shp"

gridnet= gpd.read_file(shp_file_path, engine="pyogrio")
# 替换多列的空值为 0
columns_to_fill = ['CF_Hazard', 'RF_Hazard']
gridnet[columns_to_fill] = gridnet[columns_to_fill].fillna(0)

# 计算比值，如果分母为 0，则设置为最大值 100
gridnet['ORratio'] = np.where(
    gridnet['RF_Hazard'] != 0,  # 分母不为 0
    gridnet['CF_Hazard'] / gridnet['RF_Hazard'],  # 正常计算比值
    100  # 如果分母为 0，设置为最大值 100
)

# 将 ORratio 中的值限制在 [0, 100] 范围内
gridnet['ORratio'] = np.clip(gridnet['ORratio'], 0, 100)

gridnet['CCFRtake'] = (gridnet['CCFH_qtl']* gridnet['GDP_qtl']) ** (1/2) 
gridnet['CCFRaverse'] = (gridnet['CCFH_qtl']* gridnet['pop_qtl'] * gridnet['G_capinv_q'] ) ** (1/3)  # 开三次方
gridnet['TF_Hazard'] = gridnet['TF_Hazard'].replace(0, np.nan)  # 将 0 替换为 NaN

print("风险值计算完成")

# 查看矢量的前5行/投影/列名/行数
# print(gridnet.head())
print(gridnet.crs)
print(gridnet.columns)  
print(len(gridnet))


风险值计算完成
EPSG:4326
Index(['A_CFrp0005', 'A_CFrp0010', 'A_CFrp0025', 'A_CFrp0050', 'A_CFrp0100',
       'A_CFrp0250', 'A_CFrp0500', 'A_CFrp1000', 'A_RFrp0005', 'A_RFrp0010',
       'A_RFrp0025', 'A_RFrp0050', 'A_RFrp0100', 'A_RFrp0250', 'A_RFrp0500',
       'A_RFrp1000', 'D_CFrp0005', 'D_CFrp0010', 'D_CFrp0025', 'D_CFrp0050',
       'D_CFrp0100', 'D_CFrp0250', 'D_CFrp0500', 'D_CFrp1000', 'D_RFrp0005',
       'D_RFrp0010', 'D_RFrp0025', 'D_RFrp0050', 'D_RFrp0100', 'D_RFrp0250',
       'D_RFrp0500', 'D_RFrp1000', 'FDD_km-1', 'FH_rp0005', 'FH_rp0010',
       'FH_rp0025', 'FH_rp0050', 'FH_rp0100', 'FH_rp0250', 'FH_rp0500',
       'FH_rp1000', 'MangA_km2', 'T2m', 'T_rivLC_km', 'Tp', 'V_CFrp0005',
       'V_CFrp0010', 'V_CFrp0025', 'V_CFrp0050', 'V_CFrp0100', 'V_CFrp0250',
       'V_CFrp0500', 'V_CFrp1000', 'V_RFrp0005', 'V_RFrp0010', 'V_RFrp0025',
       'V_RFrp0050', 'V_RFrp0100', 'V_RFrp0250', 'V_RFrp0500', 'V_RFrp1000',
       'area_km2', 'area_m2', 'dem', 'net_ID', 'saltA_km2', 'sla', 'wi

In [39]:


# 准备数据（假设 gridnet 是你的 DataFrame，且 GDP>0）
df = gridnet[gridnet["GDP"] > 0].copy()

# 取对数 GDP（或者用原 GDP，根据你拟合效果决定）
df["log_GDP"] = np.log10(df["GDP"]+1)

# 构造二次项
df["log_GDP_sq"] = df["log_GDP"] ** 2

# 自变量，加入常数项
X = df[["log_GDP", "log_GDP_sq"]]
X = sm.add_constant(X)

# 因变量
y = df["TF_Hazard"]

# 建立并拟合 95% 分位数回归模型
model = sm.QuantReg(y, X)
res = model.fit(q=0.999)

# 构造预测数据（细分 log_GDP）
log_gdp_grid = np.linspace(df["log_GDP"].min(), df["log_GDP"].max(), 100)
X_pred = pd.DataFrame({
    "const": 1,
    "log_GDP": log_gdp_grid,
    "log_GDP_sq": log_gdp_grid ** 2
})

# 预测 95% 分位数值
y_pred = res.predict(X_pred)

coef = res.params
print(f"拟合公式：")
print(f"TF_Hazard = {coef['const']:.4f} + {coef['log_GDP']:.4f} * log10(GDP+1) + {coef['log_GDP_sq']:.4f} * (log10(GDP+1))^2")
# 1. 计算 log_GDP 和平方项
df['log_GDP'] = np.log10(df['GDP'] + 1)

# 2. 提取拟合系数
coef = res.params

# 3. 计算拟合的上包络函数值
df['f_upper'] = (coef['const'] +
                 coef['log_GDP'] * df['log_GDP'] +
                 coef['log_GDP_sq'] * df['log_GDP']**2)

# 4. 处理 f_upper 小于等于0的异常，避免除零或负值
df.loc[df['f_upper'] <= 0, 'f_upper'] = np.nan

# 5. 计算 vulnerability
df['vulnerability'] = df['TF_Hazard'] / df['f_upper']

# 6. 将超过1的 vulnerability 强制设为1
df.loc[df['vulnerability'] > 1, 'vulnerability'] = 1

# 按倒数GDP列计算10分位数分组
df['Vul_qtl'] = pd.qcut(df['vulnerability'], 10, labels=np.linspace(0.1, 1, 10))
df['Vul_qtl'] = df['Vul_qtl'].astype(float)

print(df[['Vul_qtl']].head())

df['CCFR'] = (df['CCFH_qtl']* df['pop_qtl'] * df['Vul_qtl'] ) ** (1/3)  # 开三次方
# 7. 查看结果
print(df.columns)
print(len(df))

拟合公式：
TF_Hazard = 2859277.5598 + 12396902.9308 * log10(GDP+1) + -2780591.3663 * (log10(GDP+1))^2
   Vul_qtl
1      0.3
2      0.1
5      0.5
6      0.1
7      0.1
Index(['A_CFrp0005', 'A_CFrp0010', 'A_CFrp0025', 'A_CFrp0050', 'A_CFrp0100',
       'A_CFrp0250', 'A_CFrp0500', 'A_CFrp1000', 'A_RFrp0005', 'A_RFrp0010',
       'A_RFrp0025', 'A_RFrp0050', 'A_RFrp0100', 'A_RFrp0250', 'A_RFrp0500',
       'A_RFrp1000', 'D_CFrp0005', 'D_CFrp0010', 'D_CFrp0025', 'D_CFrp0050',
       'D_CFrp0100', 'D_CFrp0250', 'D_CFrp0500', 'D_CFrp1000', 'D_RFrp0005',
       'D_RFrp0010', 'D_RFrp0025', 'D_RFrp0050', 'D_RFrp0100', 'D_RFrp0250',
       'D_RFrp0500', 'D_RFrp1000', 'FDD_km-1', 'FH_rp0005', 'FH_rp0010',
       'FH_rp0025', 'FH_rp0050', 'FH_rp0100', 'FH_rp0250', 'FH_rp0500',
       'FH_rp1000', 'MangA_km2', 'T2m', 'T_rivLC_km', 'Tp', 'V_CFrp0005',
       'V_CFrp0010', 'V_CFrp0025', 'V_CFrp0050', 'V_CFrp0100', 'V_CFrp0250',
       'V_CFrp0500', 'V_CFrp1000', 'V_RFrp0005', 'V_RFrp0010', 'V_RFrp0025',
  

/public/home/jiaqizhang/anaconda/ENTER/envs/geo/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [40]:
gridnet = gridnet.merge(df[["net_ID", "CCFR","vulnerability"]], on="net_ID", how="left")

import pandas as pd
import numpy as np

def compute_weighted_sum(df, cols):
    """
    精确计算：避免 Series 加法中因首列为 0 或 NaN 导致全为 0 的问题。
    加权不归一，NaN自动跳过。
    """
    # 提取 rp 值（最后四位）并计算倒数权重
    rp_values = [int(col[-4:]) for col in cols]
    weights = [1 / rp for rp in rp_values]

    # 初始化一个全为 0.0 的 Series，与 df 的索引对齐
    result = pd.Series(0.0, index=df.index)

    # 逐列加权累加
    for col, w in zip(cols, weights):
        # 如果列缺失或全 NaN，跳过
        if col not in df.columns:
            continue
        result = result.add(df[col].fillna(0) * w, fill_value=0)

    return result

# ================================
# 示例应用：在 gridnet 中添加 D_prob
# ================================
# 所有 D_CFrp + D_RFrp 列
cols = [
    'D_CFrp0005', 'D_CFrp0010', 'D_CFrp0025', 'D_CFrp0050',
    'D_CFrp0100', 'D_CFrp0250', 'D_CFrp0500', 'D_CFrp1000',
    'D_RFrp0005', 'D_RFrp0010', 'D_RFrp0025', 'D_RFrp0050',
    'D_RFrp0100', 'D_RFrp0250', 'D_RFrp0500', 'D_RFrp1000'
]

# 替换 0 为 NaN
gridnet[cols] = gridnet[cols].replace(0, np.nan)

# 拆分为 CF 和 RF 两组列
cols_cf = [col for col in cols if 'CFrp' in col]
cols_rf = [col for col in cols if 'RFrp' in col]

# 分别计算加权和
gridnet['D_CF_prob'] = compute_weighted_sum(gridnet, cols_cf)
gridnet['D_RF_prob'] = compute_weighted_sum(gridnet, cols_rf)

# 如果需要总和
gridnet['D_prob'] = gridnet['D_CF_prob'] + gridnet['D_RF_prob']

# 输出列名确认
print(gridnet.columns)



Index(['A_CFrp0005', 'A_CFrp0010', 'A_CFrp0025', 'A_CFrp0050', 'A_CFrp0100',
       'A_CFrp0250', 'A_CFrp0500', 'A_CFrp1000', 'A_RFrp0005', 'A_RFrp0010',
       'A_RFrp0025', 'A_RFrp0050', 'A_RFrp0100', 'A_RFrp0250', 'A_RFrp0500',
       'A_RFrp1000', 'D_CFrp0005', 'D_CFrp0010', 'D_CFrp0025', 'D_CFrp0050',
       'D_CFrp0100', 'D_CFrp0250', 'D_CFrp0500', 'D_CFrp1000', 'D_RFrp0005',
       'D_RFrp0010', 'D_RFrp0025', 'D_RFrp0050', 'D_RFrp0100', 'D_RFrp0250',
       'D_RFrp0500', 'D_RFrp1000', 'FDD_km-1', 'FH_rp0005', 'FH_rp0010',
       'FH_rp0025', 'FH_rp0050', 'FH_rp0100', 'FH_rp0250', 'FH_rp0500',
       'FH_rp1000', 'MangA_km2', 'T2m', 'T_rivLC_km', 'Tp', 'V_CFrp0005',
       'V_CFrp0010', 'V_CFrp0025', 'V_CFrp0050', 'V_CFrp0100', 'V_CFrp0250',
       'V_CFrp0500', 'V_CFrp1000', 'V_RFrp0005', 'V_RFrp0010', 'V_RFrp0025',
       'V_RFrp0050', 'V_RFrp0100', 'V_RFrp0250', 'V_RFrp0500', 'V_RFrp1000',
       'area_km2', 'area_m2', 'dem', 'net_ID', 'saltA_km2', 'sla', 'windspeed',
       '

In [41]:
# ===============================
# 🗂 空间连接与中位数计算
# ===============================

# 步骤 1️⃣：使用空间连接将每个网格分配到其所在的子国家区域（subcountry）
# 这里假设 gridnet 是包含 CCFR、GDP、人口等指标的网格数据
# subcountry 是各国的省/州级行政边界数据（包含 'code_hasc' 字段）
joined = gpd.sjoin(gridnet, subcountry, how="left", predicate="intersects")

# 步骤 2️⃣：查看连接后的网格总数量，确保操作成功
print(f"连接后的网格数量: {len(joined)}")

# 步骤 3️⃣：按子国家（code_hasc）分组，计算以下变量的中位数
# - CCFR: 复合洪水风险
# - population: 人口
# - GDP: 经济产出
# 你希望替换 0 的列（中位数计算不希望 0 干扰）
cols_replace_zeros = [
    'CCFRtake', 'CCFRaverse', 'CCFR',
    'TF_Hazard', 'ORratio','D_prob', 'D_RF_prob','D_CF_prob','CF_Hazard', 'RF_Hazard',
    'vulnerability', 'population', 'GDP','A_CFrp1000'
]

# 将 0 替换为 NaN（inplace 修改 joined）
joined[cols_replace_zeros] = joined[cols_replace_zeros].replace(0, np.nan)

subcountry_stats = (
    joined.groupby('code_hasc')
    .agg({
        'CCFRtake': 'median',
        'CCFRaverse': 'median',
        'CCFR': 'median',
        'TF_Hazard': 'median',
        'CF_Hazard': 'median',
        'RF_Hazard': 'median',
        'ORratio': 'median',
        'D_prob': ['median', 'max'],
        'D_CF_prob':['median', 'max'],
        'D_RF_prob':['median', 'max'],
        'vulnerability': 'median',
        'population': 'median',
        'GDP': 'median',
        'A_CFrp1000': 'sum'
    })
    .reset_index()
)


# 展平列名（否则是多层列名）
subcountry_stats.columns = ['_'.join(col).strip('_') for col in subcountry_stats.columns]

print(subcountry_stats.columns)

# 步骤 4️⃣：保留子国家原始几何信息，用于后续合并和可视化
subcountry_keep = subcountry[['geometry', 'code_hasc','admin','name_en']]

# 步骤 5️⃣：将计算好的中位数数据合并回原始子国家图层
# 最终每个子国家都有一个中位 CCFR、人口、GDP 值
subcountry_keep = subcountry_keep.merge(subcountry_stats, on='code_hasc', how='left')

# 将0值替换为NaN
cols_to_check = ['CCFRtake_median', 'CCFRaverse_median', 'CCFR_median']
subcountry_keep[cols_to_check] = subcountry_keep[cols_to_check].replace(0, np.nan)

# 删除三列同时为NaN的行
subcountry_keep = subcountry_keep.dropna(subset=cols_to_check, how='all')
# 除以 10,单位转化为m
subcountry_keep['D_prob_median'] = subcountry_keep['D_prob_median'] / 10
subcountry_keep['D_RF_prob_median'] = subcountry_keep['D_RF_prob_median'] / 10
subcountry_keep['D_CF_prob_median'] = subcountry_keep['D_CF_prob_median'] / 10
subcountry_keep['D_prob_max'] = subcountry_keep['D_prob_max'] / 10
subcountry_keep['D_RF_prob_max'] = subcountry_keep['D_RF_prob_max'] / 10
subcountry_keep['D_CF_prob_max'] = subcountry_keep['D_CF_prob_max'] / 10

print(f"沿海subcountry的网格数量: {len(subcountry_keep)}")
# 步骤 6️⃣：将结果保存为 Excel 文件，便于分析或绘图
output_path = 'Subcountry data.xlsx'
subcountry_keep.to_excel(output_path, index=False)

print(f"✅ 结果已保存至: {output_path}")



连接后的网格数量: 49237
Index(['code_hasc', 'CCFRtake_median', 'CCFRaverse_median', 'CCFR_median',
       'TF_Hazard_median', 'CF_Hazard_median', 'RF_Hazard_median',
       'ORratio_median', 'D_prob_median', 'D_prob_max', 'D_CF_prob_median',
       'D_CF_prob_max', 'D_RF_prob_median', 'D_RF_prob_max',
       'vulnerability_median', 'population_median', 'GDP_median',
       'A_CFrp1000_sum'],
      dtype='object')
沿海subcountry的网格数量: 1528
✅ 结果已保存至: Subcountry data.xlsx
